In [ ]:
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)
from numpy.random import RandomState
from braindecode.torch_ext.util import np_to_var
from autodiag.perturbation import combine_covs, combine_vars

In [ ]:
rng = RandomState(39483498)
a = rng.randn(300,50,12)
b = rng.randn(300,10,)

In [ ]:
from braindecode.util import wrap_reshape_apply_fn, cov


In [ ]:

full_cov = wrap_reshape_apply_fn(cov, a,b, 0,0)

In [ ]:
part_covs = [wrap_reshape_apply_fn(cov, a[i:i+150],b[i:i+150], 0,0) for i in range(0,300,150)]

In [ ]:
cov_1, cov_2 = part_covs

In [ ]:
part_means = [(np.mean(a[i:i+150], axis=0) ,np.mean(b[i:i+150], axis=0)) for i in range(0,300,150)]

In [ ]:
(mean_1_a, mean_1_b), (mean_2_a, mean_2_b) = part_means

In [ ]:
full_cov.shape

In [ ]:
np.mean(np.abs((cov_1 + cov_2) /2.0 - full_cov))

In [ ]:
np.mean(np.abs(combine_covs(cov_1, 150, mean_1_a, mean_1_b, 
                            cov_2, 150, mean_2_a, mean_2_b) - full_cov))

In [ ]:
part_vars = [np.var(a[i:i+150], axis=0, ddof=1) for i in range(0,300,150)]


In [ ]:
var_1, var_2 = part_vars

In [ ]:
full_var = np.var(a, axis=0, ddof=1)

In [ ]:
np.mean(np.abs((var_1 + var_2) / 2.0 - full_var))

In [ ]:
np.mean(combine_vars(var_1, 150,mean_1_a, var_2, 150, mean_2_a) - full_var)

In [ ]:
from hyperoptim.rerun import rerun_exp

In [ ]:
import os.path

folder = 'data/models/pytorch/auto-diag/visualize-dummy/5/'
filename = os.path.join(folder, 'exp_file.py')
content = open(filename, 'r').read()

if not "save_torch_artifact(ex, exp.model.state_dict(), 'model_params.pkl')\n    return exp" in content:
    "save_torch_artifact(ex, exp.model.state_dict(), 'model_params.pkl')\n\n" in content 
    content = content.replace("save_torch_artifact(ex, exp.model.state_dict(), 'model_params.pkl')\n",
                             "save_torch_artifact(ex, exp.model.state_dict(), 'model_params.pkl')\n    return exp")

open(filename, 'w').write(content)

In [ ]:
ex = rerun_exp(folder, update_params=dict(only_return_exp=True), 
          save_experiment=False)

In [ ]:
exp = ex.result

In [ ]:
X, y  = exp.dataset.load()

In [ ]:
import torch as th
from braindecode.torch_ext.util import var_to_np, np_to_var


exp.model.load_state_dict(th.load(os.path.join(folder, 'model_params.pkl')))
exp.model.eval();

In [ ]:
# now please verify , then you can do visualization stuffs...

In [ ]:
train_set, valid_set, test_set = exp.splitter.split(X,y)

In [ ]:
train_batches = list(exp.iterator.get_batches(train_set, shuffle=False))
train_X_batches = np.concatenate(list(zip(*train_batches))[0])

### checking accuracies

In [ ]:
monitor = exp.monitors[2]

In [ ]:
train_preds = [var_to_np(exp.model(np_to_var(b[0]).cuda())) for b in train_batches]

monitor.monitor_set('train', train_preds, None,None,None, train_set)

In [ ]:
valid_batches = list(exp.iterator.get_batches(valid_set, shuffle=False))
valid_preds = [var_to_np(exp.model(np_to_var(b[0]).cuda())) for b in valid_batches]

monitor.monitor_set('valid', valid_preds, None,None,None, valid_set)

In [ ]:
test_batches = list(exp.iterator.get_batches(test_set, shuffle=False))
test_preds = [var_to_np(exp.model(np_to_var(b[0]).cuda())) for b in test_batches]

monitor.monitor_set('test', test_preds, None,None,None, test_set)

## check validity by manually decreasing beta

In [ ]:
from torch import nn
model_without_softmax = nn.Sequential()
for name, module in exp.model.named_children():
    if name == 'softmax':
        break
    model_without_softmax.add_module(name, module)

pred_fn = lambda x: var_to_np(
    th.mean(model_without_softmax(np_to_var(x).cuda()), dim=2)[:, :, 0, 0])

20-30 and 5-15

In [ ]:
train_preds = [var_to_np(model_without_softmax(np_to_var(b[0]).cuda())) for b in train_batches]


In [ ]:
train_pred_arr = np.mean(np.concatenate(train_preds, axis=0), axis=2)

In [ ]:
train_y_batches = np.concatenate(list(zip(*train_batches))[1])
abnorm_mask = train_y_batches == 1

In [ ]:
abnorm_preds = train_pred_arr[abnorm_mask,:,0]

In [ ]:
some_abnorm_X = train_X_batches[abnorm_mask][:30]

In [ ]:
orig_preds = abnorm_preds[:30]
plt.plot(orig_preds)

In [ ]:
from numpy.random import RandomState
from braindecode.visualization.perturbation import _amplitude_phase_to_complex
from braindecode.datautil.iterators import get_balanced_batches
from braindecode.util import wrap_reshape_apply_fn, corr

In [ ]:

examples = some_abnorm_X
fft_input = np.fft.rfft(examples, axis=2)

In [ ]:
amps = np.abs(fft_input)
phases = np.angle(fft_input)
perturbation = np.zeros(amps.shape, dtype=np.float32)

In [ ]:
freqs = np.fft.rfftfreq(examples.shape[2],1/100.0)
perturbation[:,:,np.searchsorted(freqs, 20):np.searchsorted(freqs,30)+1] = 0
perturbation[:,:,np.searchsorted(freqs, 5):np.searchsorted(freqs,15)+1] = 1

In [ ]:
perturbation = np.maximum(-amps, perturbation)
new_amps = amps + perturbation

In [ ]:
new_complex = _amplitude_phase_to_complex(new_amps, phases)

In [ ]:
new_in = np.fft.irfft(new_complex, axis=2).astype(np.float32)

In [ ]:
new_preds = np.mean(var_to_np(model_without_softmax(np_to_var(new_in).cuda())), axis=2)[:,:,0]

In [ ]:
preds_diff = new_preds - orig_preds

In [ ]:
plt.plot(preds_diff)

### continue

In [ ]:
from torch import nn
model_without_softmax = nn.Sequential()
for name, module in exp.model.named_children():
    if name =='softmax':
        break
    model_without_softmax.add_module(name, module)

In [ ]:
pred_fn = lambda x: var_to_np(th.mean(model_without_softmax(np_to_var(x).cuda()), dim=2)[:,:,0,0])

In [ ]:
from autodiag.perturbation import compute_amplitude_prediction_correlations

In [ ]:
%%time
amp_pred_corrs, orig_accuracy, new_accuracies  = compute_amplitude_prediction_correlations(
    pred_fn, train_X_batches[:300], n_iterations=20,
                                         batch_size=32,original_y=train_y_batches[:300])

In [ ]:
mean_corr = np.mean(amp_pred_corrs, axis=0)


fs = 100.0
freqs = np.fft.rfftfreq(train_X_batches.shape[2], d=1.0/fs)


POS_APPROX = ('angle',
    ('A1', (-5.0, 0.0)),
    ('A2', (5.0, 0.0)),
    ('Fp1', (-3.5, 3.5)),
    ('Fp2', (3.5, 3.5)),
    ('F7', (-4.0, 2.0)),
    ('F3', (-2.0, 2.0)),
    ('Fz', (0.0, 2.0)),
    ('F4', (2.0, 2.0)),
    ('F8', (4.0, 2.0)),
    ('C3', (-2.0, 0.0)),
    ('Cz', (0.0, 0.0)),
    ('C4', (2.0, 0.0)),
    ('T3', (-3,0)),
    ('T4', (3,0)),
    ('P3', (-2.0, -2.0)),
    ('Pz', (0.0, -2.0)),
    ('P4', (2.0, -2.0)),
    ('T5', (-3.5,-2.)),
    ('T6', (3.5,-2.)),          
    ('O1', (-4, -3.5)),
    ('O2', (4, -3.5)))

from braindecode.datasets.sensor_positions import get_channelpos, CHANNEL_10_20_APPROX
ch_names = ['A1', 'A2', 'C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1',
                    'FP2', 'FZ', 'O1', 'O2',
                    'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
positions = [get_channelpos(name, POS_APPROX) for name in ch_names]
positions = np.array(positions)


In [ ]:
from braindecode.visualization.plot import ax_scalp

start_freq = 14
stop_freq = 30

i_start = np.searchsorted(freqs,start_freq)
i_stop = np.searchsorted(freqs, stop_freq) + 1

freq_corr = np.mean(mean_corr[:,i_start:i_stop], axis=1)
max_abs_val = np.max(np.abs(freq_corr))
fig, axes = plt.subplots(1, 2)
class_names = ['Normal', 'Abnormal']
for i_class in range(2):
    ax = axes[i_class]
    ax_scalp(freq_corr[:,i_class], ch_names, chan_pos_list=POS_APPROX, cmap=cm.coolwarm,
            vmin=-max_abs_val, vmax=max_abs_val, ax=ax,annotate=True)
    ax.set_title(class_names[i_class])


start_freq = 7
stop_freq = 14

i_start = np.searchsorted(freqs,start_freq)
i_stop = np.searchsorted(freqs, stop_freq) + 1

freq_corr = np.mean(mean_corr[:,i_start:i_stop], axis=1)
from braindecode.visualization.plot import ax_scalp

fig, axes = plt.subplots(1, 2)
class_names = ['Normal', 'Abnormal']
for i_class in range(2):
    ax = axes[i_class]
    ax_scalp(freq_corr[:,i_class], ch_names, chan_pos_list=POS_APPROX, cmap=cm.coolwarm,
            vmin=-max_abs_val, vmax=max_abs_val, ax=ax,annotate=True)
    ax.set_title(class_names[i_class])

In [ ]:
from braindecode.util import cov

In [ ]:
from autodiag.perturbation import compute_amplitude_prediction_correlations_batchwise

In [ ]:
%%time
#from braindecode.visualization.perturbation import compute_amplitude_prediction_correlations
amp_pred_corrs, orig_accuracy, new_accuracies  = compute_amplitude_prediction_correlations_batchwise(
    pred_fn, train_X_batches[:300], n_iterations=20,
                                         batch_size=32,original_y=train_y_batches[:300])

In [ ]:
mean_corr = np.mean(amp_pred_corrs, axis=0)

In [ ]:
from braindecode.visualization.plot import ax_scalp

start_freq = 14
stop_freq = 30

i_start = np.searchsorted(freqs,start_freq)
i_stop = np.searchsorted(freqs, stop_freq) + 1

freq_corr = np.mean(mean_corr[:,i_start:i_stop], axis=1)
max_abs_val = np.max(np.abs(freq_corr))
fig, axes = plt.subplots(1, 2)
class_names = ['Normal', 'Abnormal']
for i_class in range(2):
    ax = axes[i_class]
    ax_scalp(freq_corr[:,i_class], ch_names, chan_pos_list=POS_APPROX, cmap=cm.coolwarm,
            vmin=-max_abs_val, vmax=max_abs_val, ax=ax,annotate=True)
    ax.set_title(class_names[i_class])


start_freq = 7
stop_freq = 14

i_start = np.searchsorted(freqs,start_freq)
i_stop = np.searchsorted(freqs, stop_freq) + 1

freq_corr = np.mean(mean_corr[:,i_start:i_stop], axis=1)
from braindecode.visualization.plot import ax_scalp

fig, axes = plt.subplots(1, 2)
class_names = ['Normal', 'Abnormal']
for i_class in range(2):
    ax = axes[i_class]
    ax_scalp(freq_corr[:,i_class], ch_names, chan_pos_list=POS_APPROX, cmap=cm.coolwarm,
            vmin=-max_abs_val, vmax=max_abs_val, ax=ax,annotate=True)
    ax.set_title(class_names[i_class])

In [ ]:
plt.plot(freqs, np.mean(mean_corr[:,:,1], axis=0))

In [ ]:
train_y_batches = np.concatenate(list(zip(*train_batches))[1])

In [ ]:
normal_batches = train_X_batches[train_y_batches == 0]

In [ ]:
abnormal_batches = train_X_batches[train_y_batches == 1]

In [ ]:
normal_bps = np.abs(np.fft.rfft(normal_batches, axis=2))

In [ ]:
abnormal_bps = np.abs(np.fft.rfft(abnormal_batches, axis=2))

In [ ]:
freqs.shape

In [ ]:
normal_bps.shape

In [ ]:
plt.plot(freqs, np.log(np.median(abnormal_bps, axis=(0,1,3)) / np.median(normal_bps, axis=(0,1,3)) ))

In [ ]:
plt.plot(freqs, np.log(np.median(normal_bps, axis=(0,3)) / np.median(abnormal_bps, axis=(0,3))).T);

In [ ]:
mean_corr.shape

In [ ]:
plt.plot(freqs, mean_corr[:,:,1].T);

In [ ]:
plt.plot(freqs, np.median(normal_bps, axis=(0,1,3)))
plt.plot(freqs, np.median(abnormal_bps, axis=(0,1,3)))

### search suitable experiment

In [ ]:
from hyperoptim.results import load_data_frame, remove_columns_with_same_value
import pandas as pd
df = load_data_frame('data/models/pytorch/auto-diag/10-fold/')
df = df[df.finished == 1]
df = df.fillna('-')
df = df.drop('seed', axis=1)
df = remove_columns_with_same_value(df)
df.runtime = pd.to_timedelta(np.round(df.runtime), unit='s')
df = remove_columns_with_same_value(df)
print(len(df))
df = df.rename(columns=dict(shrink_the_spikes='shrink',
                            channel_standardize='chan_std',
                            low_cut_hz='low_hz',
                            max_abs_val='max_val',
                            high_cut_hz='high_hz',
                            moving_demean='m_mean',
                            moving_standardize='m_std')).drop(['train_loss', 'valid_loss',
                                                       'test_loss', 'valid_misclass',
                                                       ], axis=1)
df.test_misclass = df.test_misclass * 100

#df.sort_values(by='test_sample_misclass')


In [ ]:
reduced_df = df[(df.model_constraint == 'defaultnorm') & (df.model_name == 'deep')
      & (df.max_val == 800) & (df.final_conv_length == 1)
               &(df.n_start_chans == '-')
               & (df.i_test_fold == 9)]
reduced_df

In [ ]:
reduced_df = df[(df.model_constraint == 'defaultnorm') & (df.model_name == 'shallow')
      & (df.max_val == 800) & (df.final_conv_length == 35)
               &(df.n_start_chans == '-')
               & (df.i_test_fold == 9)]
reduced_df

In [ ]:
from braindecode.visualization.perturbation import gaussian_perturbation
def compute_amplitude_prediction_correlations(pred_fn, examples, n_iterations,
                                              perturb_fn=gaussian_perturbation,
                                              batch_size=30,
                                              seed=((2017, 7, 10)),
                                              original_y=None,
                                             ):
    """
    Perturb input amplitudes and compute correlation between amplitude
    perturbations and prediction changes when pushing perturbed input through
    the prediction function.

    For more details, see [EEGDeepLearning]_.

    Parameters
    ----------
    pred_fn: function
        Function accepting an numpy input and returning prediction.
    examples: ndarray
        Numpy examples, first axis should be example axis.
    n_iterations: int
        Number of iterations to compute.
    perturb_fn: function, optional
        Function accepting amplitude array and random generator and returning
        perturbation. Default is Gaussian perturbation.
    batch_size: int, optional
        Batch size for computing predictions.
    seed: int, optional
        Random generator seed

    Returns
    -------
    amplitude_pred_corrs: ndarray
        Correlations between amplitude perturbations and prediction changes
        for all sensors and frequency bins.

    References
    ----------

    .. [EEGDeepLearning] Schirrmeister, R. T., Springenberg, J. T., Fiederer, L. D. J.,
       Glasstetter, M., Eggensperger, K., Tangermann, M., ... & Ball, T. (2017).
       Deep learning with convolutional neural networks for EEG decoding and
       visualization.
       arXiv preprint arXiv:1703.05051.
    """
    inds_per_batch = get_balanced_batches(
        n_trials=len(examples), rng=None, shuffle=False, batch_size=batch_size)
    log.info("Compute original predictions...")
    orig_preds = [pred_fn(examples[example_inds])
                  for example_inds in inds_per_batch]
    orig_preds_arr = np.concatenate(orig_preds)
    if original_y is not None:
        orig_pred_labels = np.argmax(orig_preds_arr, axis=1)
        orig_accuracy = np.mean(orig_pred_labels == original_y)
        log.info("Original accuracy: {:.2f}...".format(orig_accuracy))
    rng = RandomState(seed)
    fft_input = np.fft.rfft(examples, axis=2)
    amps = np.abs(fft_input)
    phases = np.angle(fft_input)

    amp_pred_corrs = []
    new_accuracies = []
    for i_iteration in range(n_iterations):
        log.info("Iteration {:d}...".format(i_iteration))
        log.info("Sample perturbation...")
        perturbation = perturb_fn(amps, rng)
        log.info("Compute new amplitudes...")
        # do not allow perturbation to make amplitudes go below
        # zero
        perturbation = np.maximum(-amps, perturbation)
        new_amps = amps + perturbation
        log.info("Compute new  complex inputs...")
        new_complex = _amplitude_phase_to_complex(new_amps, phases)
        log.info("Compute new real inputs...")
        new_in = np.fft.irfft(new_complex, axis=2).astype(np.float32)
        log.info("Compute new predictions...")
        new_preds = [pred_fn(new_in[example_inds])
                     for example_inds in inds_per_batch]

        new_preds_arr = np.concatenate(new_preds)
        if original_y is not None:
            new_pred_labels = np.argmax(new_preds_arr, axis=1)
            new_accuracy = np.mean(new_pred_labels == original_y)
            log.info("New accuracy: {:.2f}...".format(new_accuracy))
            new_accuracies.append(new_accuracy)
        diff_preds = new_preds_arr - orig_preds_arr

        log.info("Compute correlation...")
        amp_pred_corr = wrap_reshape_apply_fn(corr, perturbation[:, :, :, 0],
                                              diff_preds,
                                              axis_a=(0,), axis_b=(0))
        amp_pred_corrs.append(amp_pred_corr)
    if original_y is not None:
        return amp_pred_corrs, orig_accuracy, new_accuracies
    else:
        return amp_pred_corrs


def _amplitude_phase_to_complex(amplitude, phase):
    return amplitude * np.cos(phase) + amplitude * np.sin(phase) * 1j

In [ ]:
import os.path

folder = 'data/models/pytorch/auto-diag/10-fold/138/'
filename = os.path.join(folder, 'exp_file.py')
content = open(filename, 'r').read()

if not "save_torch_artifact(ex, exp.model.state_dict(), 'model_params.pkl')\n    return exp" in content:
    "save_torch_artifact(ex, exp.model.state_dict(), 'model_params.pkl')\n\n" in content 
    content = content.replace("save_torch_artifact(ex, exp.model.state_dict(), 'model_params.pkl')\n",
                             "save_torch_artifact(ex, exp.model.state_dict(), 'model_params.pkl')\n    return exp")

open(filename, 'w').write(content)

In [ ]:
ex = rerun_exp(folder, update_params=dict(only_return_exp=True), 
          save_experiment=False)

In [ ]:
exp = ex.result

In [ ]:
exp.dataset.n_recordings = 100

In [ ]:
X, y  = exp.dataset.load()


train_set, valid_set, test_set = exp.splitter.split(X,y)


import torch as th
from braindecode.torch_ext.util import var_to_np, np_to_var


exp.model.load_state_dict(th.load(os.path.join(folder, 'model_params.pkl')))
exp.model.eval();


train_batches = list(exp.iterator.get_batches(train_set, shuffle=False))
train_X_batches = np.concatenate(list(zip(*train_batches))[0])
train_y_batches = np.concatenate(list(zip(*train_batches))[1])


In [ ]:

from torch import nn
model_without_softmax = nn.Sequential()
for name, module in exp.model.named_children():
    if name =='softmax':
        break
    model_without_softmax.add_module(name, module)

pred_fn = lambda x: var_to_np(th.mean(model_without_softmax(np_to_var(x).cuda()), dim=2)[:,:,0,0])


In [ ]:

from autodiag.perturbation import compute_amplitude_prediction_correlations
amp_pred_corrs, orig_acc, new_accs = compute_amplitude_prediction_correlations(
    pred_fn, train_X_batches,  n_iterations=12, batch_size=32,original_y=train_y_batches)

In [ ]:
mean_corr = np.mean(amp_pred_corrs, axis=0)


fs = 250.0
freqs = np.fft.rfftfreq(train_X_batches.shape[2], d=1.0/fs)
start_freq = 14
stop_freq = 30

i_start = np.searchsorted(freqs,start_freq)
i_stop = np.searchsorted(freqs, stop_freq) + 1

freq_corr = np.mean(mean_corr[:,i_start:i_stop], axis=1)

POS_APPROX = CHANNEL_10_20_APPROX + (('T3', (-3,0)), ('T4', (3,0)),
                       ('T5', (-2.5,-2.5)), ('T6', (2.5,-2.5)))

from braindecode.datasets.sensor_positions import get_channelpos, CHANNEL_10_20_APPROX
ch_names = ['M1', 'M2', 'C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1',
                    'FP2', 'FZ', 'O1', 'O2',
                    'P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
positions = [get_channelpos(name, POS_APPROX) for name in ch_names]
positions = np.array(positions)

max_abs_val = np.max(np.abs(freq_corr))

In [ ]:
from braindecode.visualization.plot import ax_scalp

fig, axes = plt.subplots(1, 2, figsize=(12,8))
class_names = ['Normal', 'Abnormal']
for i_class in range(2):
    ax = axes[i_class]
    ax_scalp(freq_corr[:,i_class], ch_names, chan_pos_list=POS_APPROX, cmap=cm.coolwarm,
            vmin=-max_abs_val, vmax=max_abs_val, ax=ax,annotate=True)
    ax.set_title(class_names[i_class])

In [ ]:
np.save(os.path.join(folder, 'perturbation.npy'),
        [amp_pred_corrs,orig_acc, new_accs])

In [ ]:
a,b,c= np.load(os.path.join(folder, 'perturbation.npy'))

In [ ]:
plt.plot(c - b)